# Chapter 4 Numerical Differentiation and Integration

In [1]:
import time
import numpy as np

## Composite Simpson's Rule

Determine the value of $n$ required to approximate

$$
\int^{2}_{0} \frac{1}{x+4} dx
$$

to within $10^{-5}$ and compute the approximation by Composite Simpson's rule.

The error form for the Composite Simpson’s rule for $f(x)=\frac{1}{x+4}$ on $[0,2]$ is

$$
\left|\frac{b-a}{180}h^{4}f^{(4)}(\mu)\right|=\left|\frac{4h^{4}}{15(x+4)^5}\right|.
$$

To ensure sufficient accuracy with this technique we need to have 

$$
\frac{4h^{4}}{15}\left|\frac{1}{(x+4)^5}\right|\leq\frac{h^4}{3840}\leq10^{-5}.
$$

Using again the fact that $h=\frac{2}{n}$ gives 

$$
\frac{1}{240 n^4}\leq10^{-5} \quad \Rightarrow \quad n > \frac{10}{24^{0.25}} \approx4.5180100180492.
$$

So Composite Simpson’s rule requires only even $n\geq6$.

Composite Simpson’s rule with $n=6, h=\frac{1}{3}$ gives

$$
\int^{2}_{0} \frac{1}{x+4} dx \approx \frac{1}{9}\left(\frac{5}{12}+2\sum_{j=1}^{2}{\frac{1}{x_{2j}+4}}+4\sum_{j=1}^{3}{\frac{1}{x_{2j-1}+4}}\right)= 0.4054663746.
$$

This is accurate to within about $10^{-5}$ because the true value is

$$
\int^{2}_{0} \frac{1}{x+4} dx = \ln(\frac{3}{2}) \approx 0.4054651081.
$$

In [2]:
## Composite Simpson's Rule
def f(x):
    return 1 / (x + 4)

a = 0
b = 2
n = 6
h = (b - a) / n
I = np.log(1.5)    # true value

start = time.perf_counter()

X = f(np.linspace(a, b, n+1))
XI0 = X[0] + X[-1]
XI1 = np.sum(X[1:-1:2])    # odd sum
XI2 = np.sum(X[2:-1:2])    # even sum
XI = h * (XI0 + 2 * XI2 + 4 * XI1) / 3

elapsed = time.perf_counter() - start

error1 = abs(XI - I)
error2 = abs(XI - I) / abs(I)

print('true value=%.10f' % I)
print('approximation & %.10f \\\\' % XI)
print('absolute error & %.10f \\\\' % error1)
print('relative error & %.10f \\\\' % error2)
print('time (seconds) & %.10f' % elapsed)

true value=0.4054651081
approximation & 0.4054663746 \\
absolute error & 0.0000012665 \\
relative error & 0.0000031235 \\
time (seconds) & 0.0002828000


## Romberg Integration

Use Romberg integration to compute $R_{3,3}$ for 

$$
\int_{1}^{1.5}{x^{2}\ln x dx}.
$$

The Composite Trapezoidal rule is used to find approximations to Eq. with $n=1,2,4,\dots,32$. Then Romberg extrapolation is performed on the results.

The Composite Trapezoidal rule for the various values of n gives different approximations to the true value $0.192259$.

In [3]:
## Romberg Integration
def f(x):
    return (x ** 2) * np.log(x)

a = 1
b = 1.5
n = 3    # when n=5, error cannot be estimated
I = 1.5 ** 3 / 3 * np.log(1.5) - 1 / 9 * (1.5 ** 3 - 1)    # true value

start = time.perf_counter()

h = b - a
R = np.zeros([2, n])
R[0, 0] = h * (f(a) + f(b)) / 2
for i in range(1, n):
    temp = np.array([a+(k-.5)*h for k in range(1, 2**(i-1)+1)])
    R[1, 0] = (R[0, 0] + h * np.sum(f(temp))) / 2
    for j in range(1, i+1):
        R[1, j] = R[1, j-1] + (R[1, j-1] - R[0, j-1]) / (4 ** j - 1)
    
    h = h / 2
    R[0] = R[1]

elapsed = time.perf_counter() - start

error1 = abs(R[-1, -1] - I)
error2 = abs(R[-1, -1] - I) / abs(I)

print('true value=%.10f' % I)
print('approximation & %.10f \\\\' % R[-1, -1])
print('absolute error & %.10f \\\\' % error1)
print('relative error & %.10f \\\\' % error2)
print('time (seconds) & %.10f' % elapsed)

true value=0.1922593577
approximation & 0.1922593373 \\
absolute error & 0.0000000204 \\
relative error & 0.0000001062 \\
time (seconds) & 0.0002347000


## Composite Gaussian Quadrature

Approximate the integral in Eq. using Composite Gaussian quadrature with $n=3$.

$$
\int_{0}^{1}x^{2}\mathrm{e}^{-x} dx.
$$

Let $f(x)=x^{2}\mathrm{e}^{-x},x_{k}=kh\,(k=0,1,\dots,n)$, where $h=\frac{1}{n}$, and apply the $2$-node Gaussian-Legendre Quadrature in $[x_{k},x_{k+1}]$:

$$
\int_{x_k}^{x_{k+1}} f(x) dx \approx \frac{x_{k+1}-x_{k}}{2}\left[f\left(\frac{x_{k+1}+x_{k}}{2} + \frac{x_{k+1}-x_{k}}{2\sqrt3}\right) + f\left(\frac{x_{k+1}+x_{k}}{2} - \frac{x_{k+1}-x_{k}}{2\sqrt3}\right)\right],
$$

We have the following Composite Gauss-Legendre Quadrature in $[0,1]$ (See Eq.).

$$
\int_{0}^{1}f(x) dx = \frac{h}{2}\sum_{k=0}^{n-1}\left[f\left(x_{k+\frac{1}{2}}+\frac{h}{2\sqrt 3}\right)+f\left(x_{k+\frac{1}{2}}-\frac{h}{2\sqrt 3}\right)\right].
$$

The exact value is 
$$
\int_{0}^{1}{x^{2}\mathrm{e}^{-x} dx} = 2 - \frac{5}{\mathrm{e}} \approx 0.16060
$$

In [4]:
## Composite Gaussian Quadrature with 2-node
def f(x):
    return np.exp(-x) * x ** 2

a = 0
b = 1
n = 3
h = (b - a) / n
I = 2 - 5 / np.e   # true value

start = time.perf_counter()

X = np.linspace(a, b, n+1)[:-1] + .5 * h
X1 = f(X + h / (2 * np.sqrt(3)))
X2 = f(X - h / (2 * np.sqrt(3)))
XI = h * np.sum(X1 + X2) / 2

elapsed = time.perf_counter() - start

error1 = abs(XI - I)
error2 = abs(XI - I) / abs(I)


print('true value=%.10f' % I)
print('approximation & %.10f \\\\' % XI)
print('absolute error & %.10f \\\\' % error1)
print('relative error & %.10f \\\\' % error2)
print('time (seconds) & %.10f' % elapsed)

true value=0.1606027941
approximation & 0.1605868586 \\
absolute error & 0.0000159356 \\
relative error & 0.0000992235 \\
time (seconds) & 0.0002385000


We can also equally divided $[0,1]$ into $2n$ parts. Let $x_{k}=kh\,(k=0,1,\dots,2n)$, where $h=\frac{1}{2n}$, and use the $3$-node Gauss-Legendre Quadrature in $[x_{k}, x_{k+2}]$,

$$
\int_{x_k}^{x_{k+2}} f(x) dx \approx \frac{x_{k+2}-x_{k}}{2}\left\lbrace \frac{5}{9}\left[f\left(\frac{x_{k+2}+x_{k}}{2} + \frac{x_{k+2}-x_{k}}{2}\sqrt{\frac{3}{5}}\right) + f\left(\frac{x_{k+2}+x_{k}}{2} - \frac{x_{k+2}-x_{k}}{2}\sqrt{\frac{3}{5}}\right)\right]+\frac{8}{9}f(x_{k+1})\right\rbrace,
$$

then we have 

$$
\int_{0}^{1} f(x) dx \approx \frac{h}{9}\sum_{k=0}^{n-1}\left\lbrace 5\left[f\left(x_{2k+1} + h\sqrt{\frac{3}{5}}\right) + f\left(x_{2k+1} - h\sqrt{\frac{3}{5}}\right)\right]+8f(x_{2k+1})\right\rbrace.
$$

In [5]:
## Composite Gaussian Quadrature with 3-node
def f(x):
    return np.exp(-x) * x ** 2

a = 0
b = 1
n = 3
h = (b - a) / (2 * n)
I = 2 - 5 / np.e   # true value

start = time.perf_counter()

X = np.array([a + (2 * k + 1) * h for k in range(n)])
X1 = f(X + h * np.sqrt(0.6))
X2 = f(X - h * np.sqrt(0.6))
X3 = f(X)
XI = h * np.sum(5 * (X1 + X2) + 8 * X3) / 9

elapsed = time.perf_counter() - start

error1 = abs(XI - I)
error2 = abs(XI - I) / abs(I)

print('true value=%.10f' % I)
print('approximation & %.10f \\\\' % XI)
print('absolute error & %.10f \\\\' % error1)
print('relative error & %.10f \\\\' % error2)
print('time (seconds) & %.10f' % elapsed)

true value=0.1606027941
approximation & 0.1606027834 \\
absolute error & 0.0000000108 \\
relative error & 0.0000000671 \\
time (seconds) & 0.0002235000


## Comparison

$$
\int_{0}^{0.35} \frac{2}{x^{2}-4} dx = \frac{1}{2}\ln(\frac{2-0.35}{2+0.35}) \approx -0.17682
$$

In [6]:
## Comparison
def f(x):
    return 2 / (x ** 2 - 4)

a = 0
b = .35
I = .5 * np.log((2 - .35) / (2 + .35))    # true value

# Simpson
n = 6
h = (b - a) / n

start = time.perf_counter()

X = f(np.linspace(a, b, n+1))
XI0 = X[0] + X[-1]
XI1 = np.sum(X[1:-1:2])    # odd sum
XI2 = np.sum(X[2:-1:2])    # even sum
XI_S = h * (XI0 + 2 * XI2 + 4 * XI1) / 3

elapsed_S = time.perf_counter() - start

error1_S = abs(XI_S - I)
error2_S = abs(XI_S - I) / abs(I)

# Romberg
n = 3

start = time.perf_counter()

h = b - a
R = np.zeros([2, n])
R[0, 0] = h * (f(a) + f(b)) / 2
for i in range(1, n):
    temp = np.array([a+(k-.5)*h for k in range(1, 2**(i-1)+1)])
    R[1, 0] = (R[0, 0] + h * np.sum(f(temp))) / 2
    for j in range(1, i+1):
        R[1, j] = R[1, j-1] + (R[1, j-1] - R[0, j-1]) / (4 ** j - 1)
    
    h = h / 2
    R[0] = R[1]

elapsed_R = time.perf_counter() - start

error1_R = abs(R[-1, -1] - I)
error2_R = abs(R[-1, -1] - I) / abs(I)

## Composite Gaussian Quadrature with 2-node
n = 3
h = (b - a) / n

start = time.perf_counter()

X = np.linspace(a, b, n+1)[:-1] + .5 * h
X1 = f(X + h / (2 * np.sqrt(3)))
X2 = f(X - h / (2 * np.sqrt(3)))
XI_G2 = h * np.sum(X1 + X2) / 2

elapsed_G2 = time.perf_counter() - start

error1_G2 = abs(XI_G2 - I)
error2_G2 = abs(XI_G2 - I) / abs(I)

## Composite Gaussian Quadrature with 3-node
n = 3
h = (b - a) / (2 * n)

start = time.perf_counter()

X = np.array([a + (2 * k + 1) * h for k in range(n)])
X1 = f(X + h * np.sqrt(0.6))
X2 = f(X - h * np.sqrt(0.6))
X3 = f(X)
XI_G3 = h * np.sum(5 * (X1 + X2) + 8 * X3) / 9

elapsed_G3 = time.perf_counter() - start

error1_G3 = abs(XI_G3 - I)
error2_G3 = abs(XI_G3 - I) / abs(I)

# output results
print('true value=%.10f' % I)

print('\\textbf{} & \\textbf{Simpson} & \\textbf{Romberg} & \\textbf{Gaussian-2} & \\textbf{Gaussian-3} \\\\')
print('approximation & {0:.10f} & {1:.10f} & {2:.10f} & {3:.10f} \\\\'.format(XI_S, R[-1, -1], XI_G2, XI_G3))
print('absolute error & {0:.10f} & {1:.10f} & {2:.10f} & {3:.10f} \\\\'.format(error1_S, error1_R, error1_G2, error1_G3))
print('relative error & {0:.10f} & {1:.10f} & {2:.10f} & {3:.10f} \\\\'.format(error2_S, error2_R, error2_G2, error2_G3))
print('time (seconds) & {0:.10f} & {1:.10f} & {2:.10f} & {3:.10f}'.format(elapsed_S, elapsed_R, elapsed_G2, elapsed_G3))

true value=-0.1768200201
\textbf{} & \textbf{Simpson} & \textbf{Romberg} & \textbf{Gaussian-2} & \textbf{Gaussian-3} \\
approximation & -0.1768200398 & -0.1768200225 & -0.1768200070 & -0.1768200201 \\
absolute error & 0.0000000196 & 0.0000000023 & 0.0000000131 & 0.0000000000 \\
relative error & 0.0000001111 & 0.0000000132 & 0.0000000740 & 0.0000000000 \\
time (seconds) & 0.0002357000 & 0.0002145000 & 0.0001774000 & 0.0001648000
